In [1]:
from data_utils import load_data, preprocess_data_faiss

In [2]:
data = load_data("data")
data = preprocess_data_faiss(data)
data

/mnt/c/Users/Raman/PycharmProjects/stackexchange-recsys/data_utils.py:199: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  posts['body'] = posts['body'].map(clean_html)


,id,posttypeid,acceptedanswerid,creationdate,score,viewcount,body,owneruserid,lasteditoruserid,lasteditdate,...,title,tags,answercount,commentcount,favoritecount,parentid,communityowneddate,closeddate,ownerdisplayname,lasteditordisplayname
0,1,1,925,2010-07-09T19:07:45.460,54,8441,My chocolate chips cookies are always too cris...,16,41,2011-08-22T23:44:41.317,...,How can I get chewy chocolate chip cookies?,<baking><cookies><texture>,9,5,19,NaN,NaN,NaN,NaN,NaN
1,2,1,26,2010-07-09T19:08:34.570,29,76282,I've heard of people cooking bacon in an oven ...,17,41,2010-11-10T04:44:49.043,...,How should I cook bacon in an oven?,<oven><cooking-time><bacon>,13,3,4,NaN,NaN,NaN,NaN,NaN
2,3,1,25,2010-07-09T19:08:47.673,62,2883,"I always use brown extra large eggs, but I can...",14,1672,2013-03-24T21:42:44.057,...,What is the difference between white and brown...,<eggs>,4,1,1,NaN,NaN,NaN,NaN,NaN
3,4,1,13,2010-07-09T19:09:20.653,39,2877,And can I use one in place of the other in cer...,24,1443,2010-07-31T06:42:51.347,...,What is the difference between baking soda and...,<substitutions><please-remove-this-tag><baking...,4,0,4,NaN,NaN,NaN,NaN,NaN
4,5,1,4122,2010-07-09T19:09:59.187,48,154625,It seems that every time I make a tomato sauce...,21,41,2010-10-14T14:52:44.257,...,"In a tomato sauce recipe, how can I cut the ac...",<sauce><pasta><tomatoes><italian-cuisine>,25,1,8,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42862,55491,1,NaN,2015-03-07T17:00:51.300,0,22,I want to know if my food I cooked in my stain...,34057,NaN,NaN,...,gray film on my stainless steel stockpot food ...,<food-safety>,0,3,NaN,NaN,NaN,NaN,NaN,NaN
42866,55495,1,NaN,2015-03-07T18:25:41.330,0,13,I boil rice often and then store them. Next da...,34059,NaN,NaN,...,"When I boil rice, they lump together after a w...",<rice>,1,1,NaN,NaN,NaN,NaN,NaN,NaN
42867,55496,1,NaN,2015-03-07T18:30:56.803,0,18,"when I am cooking curry, I add chopped tomatoe...",34059,NaN,NaN,...,Why does tomato not dissolve when I make curry,<tomatoes><curry>,0,3,NaN,NaN,NaN,NaN,NaN,NaN
42878,55508,1,NaN,2015-03-07T22:16:59.250,1,18,I'm making oven baked caponata and the recipe ...,34065,NaN,NaN,...,"Definition of a ""half glass?""",<italian-cuisine>,0,1,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
import faiss
from sentence_transformers import SentenceTransformer
import numpy as np

In [5]:
model = SentenceTransformer("BAAI/bge-m3")

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/123 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/15.8k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/54.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/687 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/964 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/191 [00:00<?, ?B/s]

In [6]:
body_embeddings = model.encode(data["body"].tolist())
title_embeddings = model.encode(data["title"].tolist())

In [73]:
combined_embeddings = np.hstack((body_embeddings, title_embeddings))

In [74]:
index = faiss.IndexFlatL2(combined_embeddings.shape[1])
index.add(combined_embeddings)

In [83]:
random_embedding = combined_embeddings[
    np.random.choice(combined_embeddings.shape[0])
].reshape(1, -1)
random_embedding.shape

(1, 2048)

In [84]:
D, I = index.search(random_embedding, k=6)

In [85]:
print(f"Distances (GPU): {D}")
print(f"Indices (GPU): {I}")

Distances (GPU): [[0.        1.0208561 1.2423382 1.296716  1.3081752 1.3465927]]
Indices (GPU): [[ 6396 11036 11634  7172 11081   912]]


In [86]:
print(
    f"===ORIGINAL POST===\n"
    f"TITLE: {data['title'].iloc[I[0][0]]}\n"
    f"CONTENT: {data['body'].iloc[I[0][0]]}"
)

for i in range(1, 6):
    print(
        f"===RECOMMENDATION {i} (distance = {D[0][i]})===\n"
        f"TITLE: {data['title'].iloc[I[0][i]]}\n"
        f"CONTENT: {data['body'].iloc[I[0][i]]}"
    )

===ORIGINAL POST===
TITLE: Egg yolks vs Whole eggs
CONTENT: I'm about to make cinnamon rolls & one recipe calls for 2 whole eggs. The other recipe calls for 3 egg yolks. Which one will give me a moist, soft, cinnamon roll? 

===RECOMMENDATION 1 (distance = 1.0208561420440674)===
TITLE: Whole eggs versus folding in egg whites
CONTENT: I have a cake recipe that calls for egg yolks and separately folding in egg whites. What would be the difference in eliminating folding in the whipped egg whites and just using the whole egg? Would it work if I want a "sturdier" cake? That was my thinking but I wasn't certain of the outcome.

===RECOMMENDATION 2 (distance = 1.2423381805419922)===
TITLE: egg yolks and egg whites
CONTENT: I was making an old recipe to serve and it is with pineapple, cream cheese, dream whip and other ingredients and it says to beat 2 egg yolks and mix in then beat the 2 egg whites and fold in at no time do you cook this recipe.  You put vanilla wafers for the crust and then 